In [ ]:
%config Completer.use_jedi = False

# Generate test dataset

In [ ]:
import numpy as np

In [ ]:
# R: original matrix, P, Q: factorization matrix, K: latent element demension
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN,],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])

num_users, num_items = R.shape
K = 3

# Set size and random values of P, Q
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

# Build a single value decomposition algorithm

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
def get_rmse(R, P, Q, non_zeros):
    """Calculate and return a positive root of MSE with R and predicted R
    R: original data,
    P, Q: factorization matrix,
    non_zeros: non zero element and (x, y) coordinate of R"""

    # Generate a predicted R with P, Q
    full_pred_matrix = np.dot(P, Q.T)
    
    # Calculate RMSE with R and predicted R
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [ ]:
# Save non zeros elements and the index in the non_zeros list
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

# Update P and Q with SGD method
for step in range(steps):
    for i, j, r in non_zeros:
        # Calculate error
        eij = r - np.dot(P[i, :], Q[j, :])
        # Update P, Q with SGD and L2 regularization
        P[i, :] = P[i, :] + (learning_rate * ((eij * Q[j, :]) - (r_lambda * P[i, :])))
        Q[j, :] = Q[j, :] + (learning_rate * ((eij * P[i, :]) - (r_lambda * Q[j, :])))
    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0:
        print("### iteration step: ", step, "rmse: ", rmse)

# Predict

In [ ]:
pred_matrix = np.dot(P, Q.T)
print("A predicted matrix: \n", np.round(pred_matrix, 3))